<a href="https://colab.research.google.com/github/aaditkamat/analytics-projects/blob/main/HIV_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
df = pd.read_excel('http://www.unaids.org/sites/default/files/media_asset/HIV_estimates_from_1990-to-present.xlsx', skiprows=range(4))
df.head()

In [ ]:
percent_groups = df.iloc[0, :].dropna().unique()
estimate_levels = df.iloc[1, :].dropna().unique()
years = df.iloc[:, 0].dropna().unique().astype(np.int64)
locations = df.iloc[:, 2].dropna().unique()
vals_df = df.iloc[2: , 3: ]

In [ ]:
index_1 = pd.MultiIndex.from_tuples([(x, y) for x in percent_groups for y in estimate_levels], names=['percent_groups', 'estimate_levels'])
index_2 = pd.MultiIndex.from_tuples([(x, y) for x in years for y in locations], names=['years', 'locations'])
index_1.shape, index_2.shape

In [ ]:
vals_df.shape

In [ ]:
vals_df

In [ ]:
def replace(x):
  if 'm' in str(x):
     return int(float(x.replace('m', '')) * 1e6)
  if '...' in str(x):
    return np.nan
  if ' ' in str(x):
    return int(''.join(x.split(' ')))
  if '<' in str(x):
    return np.finfo(np.float64).eps
  return x


In [ ]:
data = np.vectorize(replace)(vals_df.values)

In [ ]:
new_df = pd.DataFrame(index=index_2, columns=index_1, data=data)
regions = ['Global', 'Asia and the Pacific', 'Eastern and southern Africa', 'Eastern Europe and central Asia'
, 'Latin America', 'Middle East and North Africa', 'Western and central Africa', 'Western and central Europe and North America']
row_indexer = [val for val in new_df.index if 2022 in val]
col_indexer = [val for val in new_df.columns
               if 'adults and children living with HIV' in val[0]
               or 'Adults and children newly infected with HIV' in val[0]
               or 'All ages incidence' in val[0]]
hiv_2022 = new_df.loc[row_indexer, col_indexer]

In [ ]:
col_indexer = [val for val in hiv_2022.columns if 'All ages incidence' in val[0]]
hiv_2022.loc[:, col_indexer] = hiv_2022.loc[:, col_indexer] * 1000

In [ ]:
hiv_2022.rename(columns={val[0]: val[0].replace('(per 1000 uninfected population)', '') for val in col_indexer}, inplace=True)
hiv_2022

In [ ]:
hiv_2022.loc[:, 'Sum'] = hiv_2022.apply(np.sum, axis=1)
hiv_2022.sort_values(by='Sum', ascending=False).iloc[: 20]